In [15]:
import os
import pandas as pd
import numpy as np
import multiprocessing as mp
import fiftyone


from loader import images as im
from itertools import repeat
from IPython.core.display import HTML

from tqdm import tqdm
tqdm.pandas()

# %pip install opencv-python-headless

ModuleNotFoundError: No module named 'fiftyone'

### Analyzing the train data

In [2]:
textcaps_train = fiftyone.zoo.load_zoo_dataset("open-images-v6", split="train")

NameError: name 'fiftyone' is not defined

In [2]:
vizwiz = vw.VizWiz(annotation_file='annotations/train.json')

loading annotations into memory...
Done (t=0.45s)
creating index...
index created! imgs = 23431, anns = 100575


In [3]:
vizwiz_df = pd.DataFrame.from_dict(vizwiz.dataset['annotations'], orient='columns')
vizwiz_images_df = pd.DataFrame.from_dict(vizwiz.dataset['images'], orient='columns')

vizwiz_df = vizwiz_df.merge(vizwiz_images_df.rename({'id': 'image_id', 'text_detected': 'image_text_detected'}, axis=1), 
                            on='image_id', how='left')

In [4]:
# caption counts for each image
vizwiz_df['captions_count'] = vizwiz_df.groupby(by=['image_id'])['caption'].transform('count')

# word counts for each caption
vizwiz_df['words_count'] = vizwiz_df['caption'].str.split().apply(len)

# https://stackoverflow.com/questions/61299310/pandas-groupby-with-custom-function-to-return-the-column-values-as-an-array

In [ ]:
%%time

# create df with unique image id and file name
cols = ['image_id', 'file_name']
vizwiz_unique = vizwiz_df.groupby(by=cols, as_index=False).first()[cols]

# find dimension for each image
bucket = 'assistive-vision'
train_imgs = 'vizwiz/train'

images = im.ImageS3(bucket)

def dimension(path, f):
    fpath = ''.join([path, '/', f])
    img = images.getImage(fpath)
    return img.shape

#vizwiz_unique['shape'] = vizwiz_unique['file_name'].progress_apply(lambda f: dimension(f))

# parallelize the tasks to available cpu cores
with mp.Pool(mp.cpu_count()) as pool:
    vizwiz_unique['shape'] = pool.starmap(dimension, zip(repeat(train_imgs), vizwiz_unique['file_name']))

In [ ]:
vizwiz_unique.drop(columns=['file_name'], inplace=True)
vizwiz_df = vizwiz_df.merge(vizwiz_unique, on='image_id', how='left')


In [ ]:
display(HTML(vizwiz_df.head().to_html()))

In [ ]:
vizwiz_df['height'], vizwiz_df['width'], _ = zip(*vizwiz_df['shape'])

### Next is the val data

In [ ]:
vizwiz_val = vw.VizWiz(annotation_file='annotations/val.json')

In [ ]:
vizwiz_val_df = pd.DataFrame.from_dict(vizwiz_val.dataset['annotations'], orient='columns')
vizwiz_images_val_df = pd.DataFrame.from_dict(vizwiz_val.dataset['images'], orient='columns')

vizwiz_val_df = vizwiz_val_df.merge(vizwiz_images_val_df.rename({'id': 'image_id', 'text_detected': 'image_text_detected'}, axis=1), 
                            on='image_id', how='left')

In [ ]:
# caption counts for each image
vizwiz_val_df['captions_count'] = vizwiz_val_df.groupby(by=['image_id'])['caption'].transform('count')

# word counts for each caption
vizwiz_val_df['words_count'] = vizwiz_val_df['caption'].str.split().apply(len)


In [ ]:
%%time

# create df with unique image id and file name
cols = ['image_id', 'file_name']
vizwiz_val_unique = vizwiz_val_df.groupby(by=cols, as_index=False).first()[cols]

# find dimension for each image
val_imgs = 'vizwiz/val'

# parallelize the tasks to available cpu cores
with mp.Pool(mp.cpu_count()) as pool:
    vizwiz_val_unique['shape'] = pool.starmap(dimension, zip(repeat(val_imgs), vizwiz_val_unique['file_name']))

In [ ]:
vizwiz_val_unique.drop(columns=['file_name'], inplace=True)
vizwiz_val_df = vizwiz_val_df.merge(vizwiz_val_unique, on='image_id', how='left')

In [ ]:
display(HTML(vizwiz_val_df.head().to_html()))

In [ ]:
vizwiz_val_df['height'], vizwiz_val_df['width'], _ = zip(*vizwiz_val_df['shape'])

### Visualizing...

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict
from matplotlib import gridspec
from commons.utils import ucb_palette

%matplotlib inline

In [ ]:
cols = ['image_id', 'captions_count', 'words_count', 'height', 'width']
vizwiz_vis = vizwiz_df[cols].copy()
vizwiz_vis['type'] = 0

vizwiz_val_vis = vizwiz_val_df[cols].copy()
vizwiz_val_vis['type'] = 1

vizwiz_vis = vizwiz_vis.append(vizwiz_val_vis)
vizwiz_vis_unique = vizwiz_vis.groupby(by=['image_id'], as_index=False).first()[['image_id', 'captions_count', 'type']]


In [ ]:
def setSpinesAlphaLineWidth(ax, top_alpha=0.25, bottom_alpha=0.25, left_alpha=0.25, right_alpha=0.25,
    top_lw=0.5, bottom_lw=0.5, left_lw=0.5, right_lw=0.5):
    
    ax.spines["top"].set_alpha(top_alpha)
    ax.spines["bottom"].set_alpha(bottom_alpha)
    ax.spines["right"].set_alpha(right_alpha)
    ax.spines["left"].set_alpha(left_alpha)
    ax.spines["top"].set_linewidth(top_lw)
    ax.spines["bottom"].set_linewidth(bottom_lw)
    ax.spines["right"].set_linewidth(right_lw)
    ax.spines["left"].set_linewidth(left_lw)

In [ ]:
fig = plt.figure(figsize=(20, 6), dpi=100)

spec = gridspec.GridSpec(ncols=2, nrows=1,
                         width_ratios=[1, 2], wspace=0.15)

#####################################
## Captions by Image               ## 
#####################################
# count images by captions_count
vizwiz_vis_caps = vizwiz_vis_unique.groupby(by=['captions_count', 'type'], as_index=False).count() \
    .rename({'image_id': 'image_count'}, axis=1)

ax = fig.add_subplot(spec[0])
x = np.arange(len(vizwiz_vis_caps.captions_count.unique()))
width = 0.4

ax.bar(x, vizwiz_vis_caps.loc[vizwiz_vis_caps['type'] == 0, 'image_count'], width=width, label='Train', 
       color=ucb_palette['berkeley_blue'])
ax.bar(x + width, vizwiz_vis_caps.loc[vizwiz_vis_caps['type'] == 1, 'image_count'], width=width, label='Val', 
       color=ucb_palette['medalist'])

# fix the axis
ax.set_xticks(x + width / 2)
ax.set_xticklabels(vizwiz_vis_caps.captions_count.unique())

# show legend
ax.legend()

# axis styling
ax.set_title('# of Captions by Image', color = ucb_palette['berkeley_blue'], y = 1.02, 
    fontsize = 11, fontweight = 'light')
ax.set_xlabel('# of captions', fontsize = 10, ha='center', color = ucb_palette['berkeley_blue'])
ax.set_ylabel('# of images', fontsize = 10, ha='center', color = ucb_palette['berkeley_blue'])
ax.tick_params(bottom=False, which='major', labelsize=8)
ax.yaxis.grid(True, color=ucb_palette['bay_fog'], ls='-.', alpha=0.3)
ax.set_axisbelow(True)
setSpinesAlphaLineWidth(ax)

# show text label
for bar in ax.patches:
    bar_value = bar.get_height()
    text = f'{bar_value:,}'
    text_x = bar.get_x() + bar.get_width() / 2
    text_y = bar.get_y() + bar_value
    bar_color = bar.get_facecolor()
    ax.text(text_x, text_y, text, ha='center', va='bottom', color=bar_color, size=8)
    
    
#####################################
## Words by Capton and Image       ## 
#####################################
# count images by words_count and type (Train or Val)
vizwiz_vis_words = vizwiz_vis[['image_id', 'words_count', 'type']].groupby(by=['words_count', 'type'], as_index=False).count() \
    .rename({'image_id': 'image_count'}, axis=1)

# insert zero words_count values for images that do not have image_count by Train (type == 0) by finding delta images between all and parts 
all = vizwiz_vis_words.words_count.unique()
parts = vizwiz_vis_words.loc[(vizwiz_vis_words.words_count.isin(all)) 
                            & (vizwiz_vis_words.type == 0), 'words_count'].to_numpy()
diff = np.setdiff1d(all, parts)
new = pd.DataFrame({'words_count': diff, 
                    'type': np.zeros(len(diff), dtype=int),
                    'image_count': np.zeros(len(diff), dtype=int)})

vizwiz_vis_words = vizwiz_vis_words.append(new)

# insert zero words_count values for images that do not have image_count by Val (type == 1) by finding delta images between all and parts 
parts = vizwiz_vis_words.loc[(vizwiz_vis_words.words_count.isin(all)) 
                            & (vizwiz_vis_words.type == 1), 'words_count'].to_numpy()
diff = np.setdiff1d(all, parts)
new = pd.DataFrame({'words_count': diff, 
                    'type': np.ones(len(diff), dtype=int),
                    'image_count': np.zeros(len(diff), dtype=int)})

vizwiz_vis_words = vizwiz_vis_words.append(new)
vizwiz_vis_words.sort_values(by=['words_count', 'type'], inplace=True)

ax = fig.add_subplot(spec[1])
x = np.arange(len(all))
width = 0.4

bar_train = vizwiz_vis_words.loc[vizwiz_vis_words['type'] == 0, 'image_count'].to_numpy()
bar_train_y = bar_train[np.argmax(bar_train)]
bar_train_x = all[np.argmax(bar_train)]

bar_val = vizwiz_vis_words.loc[vizwiz_vis_words['type'] == 1, 'image_count'].to_numpy()
bar_val_y = bar_val[np.argmax(bar_val)]
bar_val_x = all[np.argmax(bar_val)]

ax.bar(x, bar_train, width=width, label='Train', color=ucb_palette['berkeley_blue'])
ax.bar(x + width, bar_val, width=width, label='Val', 
       color=ucb_palette['medalist'])

# fix the axis
ax.set_xticks(x + width / 2)
ax.set_xticklabels(all, rotation='vertical')

# show legend
ax.legend()

# axis styling
ax.set_title('# of Words by Caption and Image', color = ucb_palette['berkeley_blue'], y = 1.02, 
    fontsize = 11, fontweight = 'light')
ax.set_xlabel('# of words', fontsize = 10, ha='center', color = ucb_palette['berkeley_blue'])
ax.set_ylabel('# of images', fontsize = 10, ha='center', color = ucb_palette['berkeley_blue'])
ax.tick_params(bottom=False, which='major', labelsize=8)
ax.yaxis.grid(True, color=ucb_palette['bay_fog'], ls='-.', alpha=0.3)
ax.set_axisbelow(True)
setSpinesAlphaLineWidth(ax)

# show text label on max value
for bar in ax.patches:
    bar_value = bar.get_height()
    if bar_value in [bar_train_y, bar_val_y]:
        text = f'({bar_train_x}, {bar_value:,})'
        if bar_value == bar_val_y:
            text = f'({bar_val_x}, {bar_value:,})'
        text_x = bar.get_x() + bar.get_width() / 2
        text_y = bar.get_y() + bar_value
        bar_color = bar.get_facecolor()
        bbox_props = dict(boxstyle='round4, pad=0.2', fc=bar_color, lw=0)
        ax.text(text_x, text_y + (0.02 * text_y), text, ha='center', va='bottom', color='w', size=8, bbox=bbox_props)
    
#fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(20, 6), dpi=100)

#####################################
## Height Image Distribution       ## 
#####################################

ax = fig.add_subplot(121)

idx = [0, 1]
labels = ['Train', 'Val']
colors = [ucb_palette['berkeley_blue'], ucb_palette['medalist']]

for i, l, c in zip(idx, labels, colors):
    subset = vizwiz_vis.loc[vizwiz_vis['type'] == i, 'height']
    sns.distplot(subset, hist=False, kde=True,
                 kde_kws = {'shade': True, 'linewidth': 1, 'color': c}, label=l)

# show legend
ax.legend()

# axis styling
ax.set_title('Height Distribution of Images', color = ucb_palette['berkeley_blue'], y = 1.02, 
    fontsize = 11, fontweight = 'light')
ax.set_xlabel('pixels', fontsize = 10, ha='center', color = ucb_palette['berkeley_blue'])
ax.set_ylabel('height density', fontsize = 10, ha='center', color = ucb_palette['berkeley_blue'])
ax.tick_params(bottom=False, which='major', labelsize=8)
ax.yaxis.grid(True, color=ucb_palette['bay_fog'], ls='-.', alpha=0.3)
ax.set_axisbelow(True)
setSpinesAlphaLineWidth(ax)

#####################################
## Width Image Distribution        ## 
#####################################

ax = fig.add_subplot(122)

for i, l, c in zip(idx, labels, colors):
    subset = vizwiz_vis.loc[vizwiz_vis['type'] == i, 'width']
    sns.distplot(subset, hist=False, kde=True,
                 kde_kws = {'shade': True, 'linewidth': 1, 'color': c}, label=l)

# show legend
ax.legend()

# axis styling
ax.set_title('Width Distribution of Images', color = ucb_palette['berkeley_blue'], y = 1.02, 
    fontsize = 11, fontweight = 'light')
ax.set_xlabel('pixels', fontsize = 10, ha='center', color = ucb_palette['berkeley_blue'])
ax.set_ylabel('width density', fontsize = 10, ha='center', color = ucb_palette['berkeley_blue'])
ax.tick_params(bottom=False, which='major', labelsize=8)
ax.yaxis.grid(True, color=ucb_palette['bay_fog'], ls='-.', alpha=0.3)
ax.set_axisbelow(True)
setSpinesAlphaLineWidth(ax)